# Defining and executing workflows

This section demonstrates how we can use the data available in our AOI to define and execute three different workflows for mapping from a data produce to a 3D canopy grid appropriate for input to QUIC-Fire.

### Workflow 1: TreeMap plot imputation

### Workflow 2: Data fusion approach between TreeMap and a high resolution canopy height produce (Meta, in this case)

### Workflow 3: Individual tree detection and segmentation from 3DEP ALS.

In [13]:
import os

os.environ["FASTFUELS_API_KEY"] = "test-api-key"

In [11]:
from fastfuels_sdk import Domain, Inventories, Grids

## Load and Define Area of Interest

First, we'll load the example GeoJSON file and create a GeoDataFrame to define our area of interest.

In [9]:
import geopandas as gpd
from pathlib import Path

# Load the GeoJSON file
geojson_path = Path("example.geojson")
gdf = gpd.read_file(geojson_path)

# Display the GeoDataFrame
print(f"Area of Interest:")
print(f"Bounds: {gdf.total_bounds}")
print(f"CRS: {gdf.crs}")
gdf

Area of Interest:
Bounds: [-114.11217537   46.8249675  -114.09545797   46.83247946]
CRS: EPSG:4326


,geometry
0,"POLYGON ((-114.09546 46.83248, -114.11218 46.8..."


## Create Domains for Each Workflow

We'll create three domains, one for each workflow:
- **Domain 1**: TreeMap plot imputation
- **Domain 2**: Data fusion (TreeMap + Meta2024 canopy height)
- **Domain 3**: Individual tree detection (not implemented yet)

In [17]:
# Create Domain 1: TreeMap plot imputation
print("Creating Domain 1: TreeMap plot imputation...")
domain_1 = Domain.from_geodataframe(
    geodataframe=gdf,
    name="Workflow 1 - TreeMap Plot Imputation",
    description="Domain for TreeMap plot imputation workflow",
)
print(f"Domain 1 ID: {domain_1.id}")
print()

# Create Domain 2: Data fusion (TreeMap + Meta2024)
print("Creating Domain 2: Data fusion (TreeMap + Meta2024)...")
domain_2 = Domain.from_geodataframe(
    geodataframe=gdf,
    name="Workflow 2 - Data Fusion",
    description="Domain for TreeMap + Meta2024 data fusion workflow",
)
print(f"Domain 2 ID: {domain_2.id}")
print()

print("Domains created successfully!")

Creating Domain 1: TreeMap plot imputation...
Domain 1 ID: 130f20e6c1124cfc9b1a255eafae095e

Creating Domain 2: Data fusion (TreeMap + Meta2024)...
Domain 2 ID: 733dd71f73624e6da18a78a0dd57322d

Domains created successfully!


## Workflow 1: TreeMap Plot Imputation

Create a tree inventory using TreeMap's nationwide coverage with default settings.

In [21]:
# Initialize Inventories for Domain 1
inventories_1 = Inventories.from_domain_id(domain_1.id)

# Create tree inventory from TreeMap
print("Creating tree inventory for Workflow 1 (TreeMap)...")
tree_inventory_1 = inventories_1.create_tree_inventory_from_treemap(
    version="2022",
)
print("Tree Inventory 1 Created. Running in background...")

Creating tree inventory for Workflow 1 (TreeMap)...
Tree Inventory 1 Created. Running in background...


## Workflow 2: Data Fusion (TreeMap + Meta2024)

Create a tree inventory using TreeMap with high-resolution Meta2024 canopy height data for improved tree height estimates.

In [22]:
# Initialize Inventories for Domain 2
inventories_2 = Inventories.from_domain_id(domain_2.id)

# Create tree inventory from TreeMap with Meta2024 canopy height
print("Creating tree inventory for Workflow 2 (TreeMap + Meta2024)...")
tree_inventory_2 = inventories_2.create_tree_inventory_from_treemap(
    version="2022",
    canopy_height_map_source="Meta2024"
)

print("Tree Inventory 2 Created. Running in background...")

Creating tree inventory for Workflow 2 (TreeMap + Meta2024)...
Tree Inventory 2 Created. Running in background...


## Workflow 3: 3DEP ALS

Create a tree inventory using 3DEP ALS data

In [23]:
# Wait for processing to complete
tree_inventory_1 = tree_inventory_1.wait_until_completed(verbose=True)
print(f"Tree Inventory 1 Status: {tree_inventory_1.status}")
print()
tree_inventory_2 = tree_inventory_2.wait_until_completed(verbose=True)
print(f"Tree Inventory 2 Status: {tree_inventory_2.status}")

Tree Inventory 1 Status: JobStatus.COMPLETED

Tree inventory has status `JobStatus.RUNNING` (5.00s)
Tree inventory has status `JobStatus.RUNNING` (10.00s)
Tree inventory has status `JobStatus.RUNNING` (15.00s)
Tree inventory has status `JobStatus.RUNNING` (20.00s)
Tree inventory has status `JobStatus.RUNNING` (25.00s)
Tree inventory has status `JobStatus.RUNNING` (30.00s)
Tree inventory has status `JobStatus.RUNNING` (35.00s)
Tree inventory has status `JobStatus.RUNNING` (40.00s)
Tree inventory has status `JobStatus.RUNNING` (45.00s)
Tree inventory has status `JobStatus.RUNNING` (50.00s)
Tree inventory has status `JobStatus.COMPLETED` (55.00s)
Tree Inventory 2 Status: JobStatus.COMPLETED


## Create Canopy Grids with Bulk Density

Now we'll create tree grids for each workflow that include the bulk density attribute derived from the tree inventories.

In [25]:
# Initialize Grids for both domains
grids_1 = Grids.from_domain_id(domain_1.id)
grids_2 = Grids.from_domain_id(domain_2.id)

# Create tree grids with bulk density from inventory
print("Creating tree grid for Workflow 1...")
tree_grid_1 = grids_1.create_tree_grid(
   attributes=["bulkDensity"],
   bulk_density={"source": "TreeInventory"}
)
print("Tree Grid 1 Created. Running in background...")
print()

print("Creating tree grid for Workflow 2...")
tree_grid_2 = grids_2.create_tree_grid(
   attributes=["bulkDensity"],
   bulk_density={"source": "TreeInventory"}
)
print("Tree Grid 2 Created. Running in background...")


Creating tree grid for Workflow 1...
Tree Grid 1 Created. Running in background...

Creating tree grid for Workflow 2...
Tree Grid 2 Created. Running in background...


In [26]:
# Wait for tree grids to complete
tree_grid_1 = tree_grid_1.wait_until_completed(verbose=True)
print(f"Tree Grid 1 Status: {tree_grid_1.status}")
print()
tree_grid_2 = tree_grid_2.wait_until_completed(verbose=True)
print(f"Tree Grid 2 Status: {tree_grid_2.status}")

Tree grid has status `JobStatus.PENDING` (5.00s)
Tree grid has status `JobStatus.PENDING` (10.00s)
Tree grid has status `JobStatus.PENDING` (15.00s)
Tree grid has status `JobStatus.RUNNING` (20.00s)
Tree grid has status `JobStatus.RUNNING` (25.00s)
Tree grid has status `JobStatus.COMPLETED` (30.00s)
Tree Grid 1 Status: JobStatus.COMPLETED

Tree Grid 2 Status: JobStatus.COMPLETED


## Export to Zarr Format

Export the grid data to zarr format for programmatic access and analysis.

In [27]:
# Create zarr exports
print("Creating zarr export for Workflow 1...")
export_1 = grids_1.create_export("zarr")
print("Zarr Export 1 Created. Running in background...")
print()

print("Creating zarr export for Workflow 2...")
export_2 = grids_2.create_export("zarr")
print("Zarr Export 2 Created. Running in background...")

Creating zarr export for Workflow 1...
Zarr Export 1 Created. Running in background...

Creating zarr export for Workflow 2...
Zarr Export 2 Created. Running in background...


In [28]:
# Wait for exports to complete
export_1 = export_1.wait_until_completed(verbose=True)
print(f"Export 1 Status: {export_1.status}")
print()
export_2 = export_2.wait_until_completed(verbose=True)
print(f"Export 2 Status: {export_2.status}")

Export has status `ExportStatus.PENDING` (5.00s)
Export has status `ExportStatus.PENDING` (10.00s)
Export has status `ExportStatus.PENDING` (15.00s)
Export has status `ExportStatus.COMPLETED` (20.00s)
Export 1 Status: ExportStatus.COMPLETED

Export 2 Status: ExportStatus.COMPLETED


## Save Zarr Files

Save the exported zarr data to local files.

In [30]:
# Save zarr files
print("Saving Workflow 1 zarr file...")
export_1.to_file("workflow_1_treemap.zarr.zip")
print(f"Saved to: workflow_1_treemap.zarr.zip")
print()

print("Saving Workflow 2 zarr file...")
export_2.to_file("workflow_2_data_fusion.zarr.zip")
print(f"Saved to: workflow_2_data_fusion.zarr.zip")
print()

print("All workflows completed successfully!")

Saving Workflow 1 zarr file...
Saved to: workflow_1_treemap.zarr.zip

Saving Workflow 2 zarr file...
Saved to: workflow_2_data_fusion.zarr.zip

All workflows completed successfully!
